In [9]:
# ============================================================
# 48音 主観評価（ratings/*.xlsx）→ 統合解析に使えるCSV群を生成
# 追加機能: master_epoch_index.csv があれば run を自動付与
# ============================================================

from pathlib import Path
import pandas as pd
import numpy as np
import re
import unicodedata

# ============================================================
# 0. パス設定（★ここだけ自分の環境に合わせて変更）
# ============================================================
ROOT_DIR = Path("/Users/shunsuke/EEG_48sounds")
DERIV_DIR = ROOT_DIR / "derivatives"

OUT_DIR = DERIV_DIR / "behavioral"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 評価Excelを置くフォルダ（ROOT直下に ratings/ を推奨）
DATA_DIR = ROOT_DIR / "ratings"


MASTER_EPOCH = DERIV_DIR / "master_epoch_index.csv"  # あれば run を付与できる

# =========================
# 1) 列定義
# =========================
ITEM_COLS = [
    "驚き","緊急感","脅威感","圧倒感",
    "接近したい気持ち","興味","没入","退屈",
]

RENAME_MAP = {
    # 基本キー
    "No": "number", "番号": "number",
    "ファイル名": "FileName", "filename": "FileName", "file_name": "FileName",
    "カテゴリ": "カテゴリー", "Category": "カテゴリー", "category": "カテゴリー",
    "提示順": "Trial", "trial": "Trial", "TRIAL": "Trial",
    "Trial(提示順)": "Trial", "Trial（提示順）": "Trial",
    # たまにある表記ゆれ
    "カテゴリ―": "カテゴリー",
}

REQUIRED_COLS_BASE = ["number","FileName","カテゴリー","Trial"]

# =========================
# 2) 便利関数
# =========================
def extract_participant_id(path: Path) -> str:
    stem = path.stem
    m = re.search(r"[Pp](\d{1,3})", stem)
    if m:
        return f"P{int(m.group(1)):02d}"
    m2 = re.search(r"(\d{1,3})", stem)
    if m2:
        return f"P{int(m2.group(1)):02d}"
    return stem

def extract_subject_and_run(path: Path) -> tuple[str, str]:
    """EEG側（②）と同じルールで subject / run を推定する。"""
    stem = path.stem
    m = re.match(r'^(\d+)[(（](\d+)[)）](.+)$', stem)
    if m:
        sub_no, ses_no, rest = m.group(1), m.group(2), m.group(3)
        rest = re.sub(r'(音刺激|主観評価|評価|ratings?)$', '', rest)
        subject = f"{sub_no}_{rest}"
        run = f"run{int(ses_no)}"
        return subject, run
    return extract_participant_id(path), "run1"


def to_int_safe(x):
    if pd.isna(x):
        return np.nan
    try:
        return int(float(x))
    except Exception:
        return np.nan

def normalize_filename(x: str) -> str:
    """HCU/音特徴と合わせるための正規化（番号接頭辞・-9dBA等除去）"""
    if pd.isna(x):
        return np.nan
    s = unicodedata.normalize("NFKC", str(x)).strip()
    s = re.sub(r"^\s*\d+\s*[:：]\s*", "", s)
    s = re.sub(r"(_-?\d+dBA.*)\.wav$", ".wav", s, flags=re.IGNORECASE)
    return s

def zscore_within_participant(x: pd.Series) -> pd.Series:
    mu = x.mean()
    sd = x.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return pd.Series(0, index=x.index)
    return (x - mu) / sd

def cronbach_alpha(df_items: pd.DataFrame) -> float:
    items = df_items.to_numpy(dtype=float)
    n_items = items.shape[1]
    if n_items <= 1:
        return np.nan
    item_vars = items.var(axis=0, ddof=1)
    total_var = items.sum(axis=1).var(ddof=1)
    if total_var == 0 or np.isnan(total_var):
        return np.nan
    return n_items / (n_items - 1) * (1 - item_vars.sum() / total_var)


# =========================
# 3) Excel読み込み
# =========================
# Excel読み込み
excel_files = sorted(DATA_DIR.glob("*.xlsx"))
print("ROOT_DIR   :", ROOT_DIR.resolve())
print("DATA_DIR   :", DATA_DIR.resolve())
print("OUT_DIR    :", OUT_DIR.resolve())
print("n_xlsx     :", len(excel_files))

if not excel_files:
    raise FileNotFoundError(f"{DATA_DIR.resolve()} に .xlsx がありません")


print("\n=== 読み込むExcelファイル一覧 ===")
for f in excel_files:
    print(" -", f.name)
print(f"合計 {len(excel_files)} ファイル（=被験者）")

all_list = []

for file_path in excel_files:
    pid = extract_participant_id(file_path)
    subject_guess, run_guess = extract_subject_and_run(file_path)

    print(f"\n=== 読み込み中: {file_path.name} -> participant={pid} ===")

    # 1) まず読む
    df = pd.read_excel(file_path)

    # 2) 列名標準化
    df = df.rename(columns=RENAME_MAP)

    # 3) Trial補完
    if "Trial" not in df.columns:
        print(f"⚠️ [WARN] {file_path.name}: Trial列が無いので行順(1..N)で補完します")
        df["Trial"] = np.arange(1, len(df) + 1)

    # 4) 型整形
    if "number" in df.columns:
        df["number"] = df["number"].map(to_int_safe)
    df["Trial"] = df["Trial"].map(to_int_safe)

    # 5) ここで participant/subject/run/trial_in_run を付与
    df["participant"] = pid
    df["subject"] = subject_guess
    df["run"] = run_guess
    df["trial_in_run"] = df["Trial"].map(to_int_safe)

    # 6) 必須列チェック
    required_cols = REQUIRED_COLS_BASE + ITEM_COLS
    missing_cols = [c for c in required_cols if c not in df.columns]
    if missing_cols:
        raise ValueError(
            f"{file_path.name} に不足している列があります: {missing_cols}\n"
            f"現在の列名: {list(df.columns)}"
        )

    # ★ここを追記：評価項目を数値に強制（文字列や空白をNaNに）
    for c in ITEM_COLS:
        df[c] = pd.to_numeric(df[c], errors="coerce")


    # 7) 抜き出し（統合に必要な列を全部残す）
    df_sub = df[required_cols + ["participant","subject","run","trial_in_run"]].copy()
    df_sub["FileName_norm"] = df_sub["FileName"].map(normalize_filename)

    # 8) 48行チェック
    if len(df_sub) != 48:
        raise ValueError(f"{file_path.name}: 行数が48ではありません（{len(df_sub)}行）")

    all_list.append(df_sub)

ratings = pd.concat(all_list, ignore_index=True)


print("\n=== 結合後の形状 ===")
print(ratings.shape)

# =========================
# 4) run を自動付与（master_epoch_index がある場合）
#   ここが “統合解析対応” の肝
# =========================
ratings["trial_in_run"] = ratings["Trial"].map(to_int_safe)

if MASTER_EPOCH.exists():
    print("\n=== master_epoch_index.csv があるので run/subject を照合して付与します ===")
    me = pd.read_csv(MASTER_EPOCH)

    needed_run = {"participant","run","trial_in_run","number"}
    if not needed_run.issubset(set(me.columns)):
        print("⚠️ [WARN] master_epoch_index に必要列が揃っていないため、run自動付与をスキップします。")
        print("必要列:", sorted(list(needed_run)))
        print("現状列:", list(me.columns))
    else:
        me["trial_in_run"] = me["trial_in_run"].map(to_int_safe)
        me["number"] = me["number"].map(to_int_safe)

        key = ["participant","trial_in_run","number"]

        cols = key + ["run"]
        has_subject = "subject" in me.columns
        if has_subject:
            cols += ["subject"]

        me_key = me[cols].copy()

        # 競合があれば監査ログ
        dup = me_key.duplicated(subset=key, keep=False)
        if dup.any():
            me_key[dup].sort_values(key + ["run"]).to_csv(
                OUT_DIR / "audit_master_epoch_duplicate_keys.csv",
                index=False, encoding="utf-8-sig"
            )
            me_key = me_key.sort_values(key + ["run"]).drop_duplicates(subset=key, keep="first")

        # merge（事故検知用に validate を付ける）
        tmp = ratings.merge(me_key, on=key, how="left", suffixes=("", "_master"), validate="many_to_one")

        tmp["run"] = tmp["run_master"].fillna(tmp["run"])
        tmp = tmp.drop(columns=["run_master"])

        if has_subject:
            tmp["subject"] = tmp["subject_master"].fillna(tmp["subject"])
            tmp = tmp.drop(columns=["subject_master"])

        ratings = tmp

        miss = int(ratings["run"].isna().sum())
        print("run が最終的に欠損の行:", miss)
else:
    print("\n※ master_epoch_index.csv が無いので、ファイル名推定の run/subject をそのまま使います。")


# =========================
# 5) 品質チェック
# =========================
print("\n=== 品質チェック ===")
print("\n各participantのTrial数（ユニーク）:")
print(ratings.groupby("participant")["Trial"].nunique())

print("\n各participantのnumber数（ユニーク）:")
print(ratings.groupby("participant")["number"].nunique())

print("\n各項目の欠損値数:")
print(ratings[ITEM_COLS].isna().sum())

print("\n各participant×runのtrial_in_run数（ユニーク）:")
print(ratings.groupby(["participant","run"])["trial_in_run"].nunique())

print("\n各participant×runのnumber数（ユニーク）:")
print(ratings.groupby(["participant","run"])["number"].nunique())

# =========================
# 6) z得点＆合成指標
# =========================
for col in ITEM_COLS:
    ratings[f"{col}_z"] = ratings.groupby(["participant","run"])[col].transform(zscore_within_participant)

AROUSAL_ITEMS = ["驚き", "緊急感", "脅威感", "圧倒感"]
AROUSAL_Z_ITEMS = [f"{c}_z" for c in AROUSAL_ITEMS]

ratings["Arousal_raw"] = ratings[AROUSAL_ITEMS].mean(axis=1)
ratings["Arousal_z"]   = ratings[AROUSAL_Z_ITEMS].mean(axis=1)

ratings["退屈_rev"] = 6 - ratings["退屈"]
ratings["退屈_z_rev"] = -ratings["退屈_z"]

APPROACH_RAW_ITEMS = ["接近したい気持ち", "興味", "没入", "退屈_rev"]
APPROACH_Z_ITEMS   = ["接近したい気持ち_z", "興味_z", "没入_z", "退屈_z_rev"]

ratings["Approach_raw"] = ratings[APPROACH_RAW_ITEMS].mean(axis=1)
ratings["Approach_z"]   = ratings[APPROACH_Z_ITEMS].mean(axis=1)

print("\n=== 合成指標サンプル ===")
print(ratings[["participant","run","Trial","number","FileName","Arousal_raw","Approach_raw","Arousal_z","Approach_z"]].head())

# =========================
# 7) 音レベル summary
# =========================
group_cols = ["number","FileName_norm","カテゴリー"]

agg_dict = {"n_participants": ("participant", "nunique")}
for col in ITEM_COLS:
    agg_dict[f"{col}_mean"] = (col, "mean")
    agg_dict[f"{col}_sd"]   = (col, "std")
for col in ["Arousal_raw","Arousal_z","Approach_raw","Approach_z"]:
    agg_dict[f"{col}_mean"] = (col, "mean")
    agg_dict[f"{col}_sd"]   = (col, "std")

sound_summary = ratings.groupby(group_cols).agg(**agg_dict).reset_index()

# =========================
# 8) 信頼性（alpha）
# =========================
alpha_arousal  = cronbach_alpha(ratings[AROUSAL_ITEMS])
alpha_approach = cronbach_alpha(ratings[["接近したい気持ち","興味","没入","退屈_rev"]])

print("\n=== Cronbach's alpha ===")
print(f"Arousal系4項目: α = {alpha_arousal:.3f}")
print(f"Approach系4項目: α = {alpha_approach:.3f}")

# =========================
# 9) 曖昧刺激抽出
# =========================
ambiguous_by_approach = sound_summary.sort_values("Approach_z_sd", ascending=False).head(10)
ambiguous_by_approach.to_csv(
    OUT_DIR / "ambiguous_sounds_by_Approach_sd_top10.csv",
    index=False, encoding="utf-8-sig"
)

neg = sound_summary[sound_summary["カテゴリー"]=="不快"].copy()
if len(neg) == 0:
    ambiguous_negative = neg.iloc[0:0]
else:
    thr = max(neg["Approach_z_mean"].quantile(0.7), 0.0)
    ambiguous_negative = neg[neg["Approach_z_mean"] >= thr].sort_values("Approach_z_mean", ascending=False).head(10)

ambiguous_negative.to_csv(
    OUT_DIR / "ambiguous_negative_sounds.csv",
    index=False, encoding="utf-8-sig"
)

# =========================
# 10) participant summary / participant×sound
# =========================
participant_summary = ratings.groupby(["participant","run"]).agg(
    Arousal_raw_mean=("Arousal_raw","mean"),
    Approach_raw_mean=("Approach_raw","mean"),
    Boredom_mean=("退屈","mean"),
    Arousal_z_mean=("Arousal_z","mean"),
    Approach_z_mean=("Approach_z","mean"),
).reset_index()

participant_sound_long = ratings.sort_values(["participant","run","trial_in_run"]).copy()

# =========================
# 11) 保存
# =========================
ratings_long_path = OUT_DIR / "ratings_long.csv"
sound_summary_path = OUT_DIR / "sound_summary.csv"

ratings.to_csv(ratings_long_path, index=False, encoding="utf-8-sig")
sound_summary.to_csv(sound_summary_path, index=False, encoding="utf-8-sig")
participant_sound_long.to_csv(OUT_DIR / "participant_sound_long.csv", index=False, encoding="utf-8-sig")
participant_summary.to_csv(OUT_DIR / "participant_summary.csv", index=False, encoding="utf-8-sig")

print("\n=== 完了 ===")
print("Trialレベル:", ratings_long_path)
print("音レベル  :", sound_summary_path)
print("participant×音:", OUT_DIR / "participant_sound_long.csv")
print("participant summary:", OUT_DIR / "participant_summary.csv")
print("曖昧音:", OUT_DIR / "ambiguous_sounds_by_Approach_sd_top10.csv")
print("不快×高Approach:", OUT_DIR / "ambiguous_negative_sounds.csv")


ROOT_DIR   : /Users/shunsuke/EEG_48sounds
DATA_DIR   : /Users/shunsuke/EEG_48sounds/ratings
OUT_DIR    : /Users/shunsuke/EEG_48sounds/derivatives/behavioral
n_xlsx     : 12

=== 読み込むExcelファイル一覧 ===
 - ratings_P01.xlsx
 - ratings_P02.xlsx
 - ratings_P03.xlsx
 - ratings_P04.xlsx
 - ratings_P05.xlsx
 - ratings_P06.xlsx
 - ratings_P07.xlsx
 - ratings_P08.xlsx
 - ratings_P09.xlsx
 - ratings_P10.xlsx
 - ratings_P11.xlsx
 - ratings_P12.xlsx
合計 12 ファイル（=被験者）

=== 読み込み中: ratings_P01.xlsx -> participant=P01 ===

=== 読み込み中: ratings_P02.xlsx -> participant=P02 ===

=== 読み込み中: ratings_P03.xlsx -> participant=P03 ===

=== 読み込み中: ratings_P04.xlsx -> participant=P04 ===

=== 読み込み中: ratings_P05.xlsx -> participant=P05 ===

=== 読み込み中: ratings_P06.xlsx -> participant=P06 ===

=== 読み込み中: ratings_P07.xlsx -> participant=P07 ===

=== 読み込み中: ratings_P08.xlsx -> participant=P08 ===

=== 読み込み中: ratings_P09.xlsx -> participant=P09 ===

=== 読み込み中: ratings_P10.xlsx -> participant=P10 ===

=== 読み込み中: ratings_P11.x